# URL Scrap

Scrap URL Today

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My\ Drive/GoFundMe/tutorial

!pip install -q scrapy
!pip install -q dateparser
!pip install -q furl
!pip install -q scrapy_random_fake_ua
!pip install -q fake_useragent

from datetime import date
today = date.today().strftime("%Y-%m-%d")

!scrapy crawl url_spider -O ./url/daily/{today}.json

Merge Daily URL into Database

In [2]:
import json
import os

# Create empty JSONArray if it doesn't exist
url_db_file = open('./url/url_db.json', 'r' if os.path.exists('./url/url_db.json') else 'w+')
daily_file = open('./url/daily/' + today + '.json')

daily_json = json.load(daily_file)
url_db_json = json.load(url_db_file) if os.stat("./url/url_db.json").st_size > 2 else []

url_id_list = [item['id'] for item in url_db_json]

# Read daily file and append to url_db_json 
found = len(daily_json)
added = 0
for item in daily_json:
    if item['id'] not in url_id_list:
        url_db_json.append(item)
        url_id_list.append(item['id'])
        added = added + 1

print(f'Found {found} URLs, added {added} URLs, duplicated {found-added} URLs')

# Write result file
with open('./url/url_db.json', "w") as url_db_file:
    url_db_file.write('[' +
            ',\n'.join(json.dumps(i) for i in url_db_json) +
            ']\n')

Found 2052 URLs, added 233 URLs, duplicated 1819 URLs


# Scrap Projects

In [7]:
print(f'Found {found} URLs, added {added} URLs, duplicated {found-added} URLs')

from datetime import date
today = date.today().strftime("%Y-%m-%d")
%cd /content/drive/My\ Drive/GoFundMe/tutorial

#-s JOBDIR=./crawls/project_spider/
#print("scrapy crawl project_spider  -O ./projects/daily/"+today+".json")
!scrapy crawl project_spider -s JOBDIR=./crawls/project_spider/ -O ./projects/daily/{today}.json

/content/drive/My Drive/GoFundMe/tutorial
2022-05-12 10:49:07 [scrapy.utils.log] INFO: Scrapy 2.6.1 started (bot: tutorial)
2022-05-12 10:49:07 [scrapy.utils.log] INFO: Versions: lxml 4.2.6.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 22.4.0, Python 3.7.13 (default, Apr 24 2022, 01:04:09) - [GCC 7.5.0], pyOpenSSL 22.0.0 (OpenSSL 3.0.3 3 May 2022), cryptography 37.0.2, Platform Linux-5.4.188+-x86_64-with-Ubuntu-18.04-bionic
2022-05-12 10:49:07 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'AUTOTHROTTLE_MAX_DELAY': 5,
 'AUTOTHROTTLE_START_DELAY': 0.2,
 'AUTOTHROTTLE_TARGET_CONCURRENCY': 0.6,
 'BOT_NAME': 'tutorial',
 'DUPEFILTER_CLASS': 'scrapy.dupefilters.BaseDupeFilter',
 'LOG_LEVEL': 'INFO',
 'NEWSPIDER_MODULE': 'tutorial.spiders',
 'RETRY_HTTP_CODES': [429],
 'SPIDER_MODULES': ['tutorial.spiders']}
2022-05-12 10:49:07 [scrapy.extensions.telnet] INFO: Telnet Password: 319e5eb1226b7402
2022-05-12 10:49:07 [scrapy.middleware] INFO:

In [6]:
!rm -rf ./crawls